Here we will try to look into cases where due to scraping errors some baseRankings are taken wrong, for example 156000 was taken as 156. Following code will list rows where big sudden fall offs take place in the data.

In [4]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('FINALDATA.csv')

# Sort the DataFrame by idOSYM and baseRanking
df = df.sort_values(by=['idOSYM', 'baseRanking'])

# Initialize an empty list to store the indices of the rows with sudden changes
sudden_changes_indices = []

# Iterate through the DataFrame
for i in range(1, len(df)):
    if df.iloc[i]['idOSYM'] == df.iloc[i-1]['idOSYM']:
        current_value = df.iloc[i]['baseRanking']
        previous_value = df.iloc[i-1]['baseRanking']
        if current_value >= 10 * previous_value or current_value <= previous_value / 10:
            sudden_changes_indices.append(i-1)
            sudden_changes_indices.append(i)

# Get the unique indices with sudden changes
sudden_changes_indices = list(set(sudden_changes_indices))

# Filter the DataFrame to include only the rows with sudden changes
sudden_changes_df = df.iloc[sudden_changes_indices]

# Save the new DataFrame to a CSV file
sudden_changes_df.to_csv('sudden_changes_only2.csv', index=False)

C:\Users\AliCe\AppData\Local\Temp\ipykernel_23836\449229291.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('FINALDATA.csv')


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('FINALDATA.csv')

# Drop rows where the quota column is 1 or 2
df = df[~df['quota'].isin([1, 2])]

# Sort the DataFrame by idOSYM, academicYear, and baseRanking
df = df.sort_values(by=['idOSYM', 'academicYear', 'baseRanking'])

# Initialize a dictionary to store the rows with sudden changes
sudden_changes_dict = {}

# Group the DataFrame by idOSYM
grouped = df.groupby(['idOSYM'])

# Iterate through each group
for idOSYM, group in grouped:
    group = group.dropna(subset=['baseRanking'])  # Drop rows where baseRanking is NaN
    if len(group) <= 1:  # Skip groups with only one value
        continue
    for i in range(len(group)):
        current_value = group.iloc[i]['baseRanking']
        # Check if the current value is at least 10 times bigger or smaller than all other values in the group
        if all(current_value >= 5 * other_value or current_value <= other_value / 5 for j, other_value in enumerate(group['baseRanking']) if i != j):
            sudden_changes_dict[idOSYM] = group.iloc[i]
            break

# Convert the dictionary to a DataFrame
sudden_changes_df = pd.DataFrame.from_dict(sudden_changes_dict, orient='index')
sudden_changes_df = sudden_changes_df[~sudden_changes_df['occupiedSlots'].isin([0])]
# Save the new DataFrame to a CSV file
sudden_changes_df.to_csv('sudden_changes_single_rowLast.csv', index=False)